In [72]:
import os
import pandas as pd
import openpyxl
from datetime import datetime

date = datetime.now().strftime("%m_%d_%Y")
#original format
# df_returns_export = pd.read_csv('returns_\\report-1714490019067.tsv', sep="\t",encoding= 'unicode_escape')
# df_returns_export = pd.read_csv('returns_\\amz_export.csv',encoding= 'unicode_escape')
# df_order_report = pd.read_csv('returns_\ca_export.csv')
# df_item_export = pd.read_excel('returns_\items_export.xlsx')

#merging files
returns_files = []
df_returns_export = pd.DataFrame()

path = 'returns_\\'

for root, directories, files in os.walk(path, topdown=True):
    for file_name in files:
        if file_name.endswith(".tsv"):
            if root.endswith("ab"):
                continue
            else:
                returns_files.append(pd.read_csv(os.path.join(root,file_name), sep="\t",encoding= 'unicode_escape'))

for return_file in returns_files:

    df_returns_export = df_returns_export.append(return_file, ignore_index=True
    )

items_files = []
df_items_export = pd.DataFrame()

path = 'returns_\\'

for root, directories, files in os.walk(path, topdown=True):
    for file_name in files:
        if file_name.endswith(".csv"):
            if root.endswith("ab"):
                continue
            else:
                items_files.append(pd.read_csv(os.path.join(root,file_name), encoding= 'unicode_escape'))

for item_file in items_files:

    df_items_export = df_items_export.append(item_file, ignore_index=True
    )

orders_files = []
df_orders_export = pd.DataFrame()

path = 'returns_\\'

for root, directories, files in os.walk(path, topdown=True):
    for file_name in files:
        if file_name.endswith(".xlsx"):
            if root.endswith("ab"):
                continue
            else:
                orders_files.append(pd.read_excel(os.path.join(root,file_name)))

for order_file in orders_files:

    df_orders_export = df_orders_export.append(order_file, ignore_index=True)


def str_join(df, sep, *cols):
    from functools import reduce
    return reduce(lambda x, y: x.astype(str).str.cat(y.astype(str), sep=sep),[df[col] for col in cols])

df_orders_export = df_orders_export.rename(columns={"Site Order ID": "Order ID"})
df_items_export = df_items_export.rename(columns={"Inventory Number": "SKU"})

inner_join = pd.merge(
    df_returns_export,
    df_orders_export,
    on='Order ID',
    how='inner'
)

final_report = pd.merge(
    inner_join,
    df_items_export,
    on='SKU',

    how='inner'
)

final_report.groupby('Tracking ID')['Merchant SKU','UPC'].agg(' '.join).reset_index()
final_report['full_order'] = str_join(final_report,',','Merchant SKU','UPC')
cols = ['Tracking ID'] + [col for col in final_report if col != 'Tracking ID']
final_report = final_report[cols].drop_duplicates(subset = ['Tracking ID'])
final_report.to_excel(r"C:\Users\sebas\Desktop\projects\pandas\returns_" + date +".xlsx", index=False, encoding='utf-8',sheet_name='Report')


# Create an new Excel file and add a worksheet.
wb = openpyxl.load_workbook(r"returns_"+ date + ".xlsx")

ws2 = wb.create_sheet(title="Scan")
                                                                                                              
scans_titles = ['Tracking Number',	'UPC/SKU',	'Status',	'Order Number',	'CA Order ID',	'Item info']
scans_columns = ['A','B','C','D','E','F']

# write headers and excel functions
for x in range(5):

    ws2[ str(scans_columns[x]) + str(1)] = scans_titles[x]
    if scans_titles[x] == 'Status':
        for i in range(2,500):

            ws2['C'+ str(i)] = '=IF(B'+str(i)+'="","",IF(ISNUMBER(SEARCH(B'+str(i)+',F'+str(i)+'))= FALSE,"WRONG RETURN",""))'
    elif scans_titles[x] =='Order Number':
        for i in range(2,500):
            ws2['E'+str(i)] = '=IFERROR(VLOOKUP(A' +str(i)+ ',Report!A:F,2,FALSE),"")'
    elif scans_titles[x] =='CA Order ID':
        for i in range(2,500):
            ws2['F'+str(i)] = '=IFERROR(VLOOKUP(A'+str(i)+',Report!A:F,4,FALSE),"")'
    elif scans_titles[x] =='Item info':
        for i in range(2,500):
            ws2['G'+str(i)] = '=IF(A'+str(i) + '="","",VLOOKUP(A'+str(i)+',Report!A:G,7,FALSE))'

# ws2.protection.sheet = True

# for col in ['A','B','C','E','F','G']:
#     for cell in ws2[col]:
#         cell.protection = WorkbookProtection(locked=False)

wb.save("returns_"+date+".xlsx")



KeyError: 'Order ID'